In [ ]:
import numpy as np

import objective_function as obj

### Chromosomes
- Value encoding --> Each gene represents xij
- Must be a non-negative integer

### Natural selection: 
1. Proportional selection --> Chance proportional to fitness function
2. Weighted relative fitness --> Chance proportional to the difference over the worst fitness
3. Rank-based selection --> Near termination, chance proportional to rank
4. Tournament selection --> Randomly select k1 parents then select k2 best parents among them

### Crossover
1. Multi-point crossover --> Switch whole sections
2. Uniform crossover --> Choose each xij from a random parent
3. Arithmetic mean crossover --> Mean of 2 numbers

### Mutation
1. Uniform mutation --> Change a single value up or down
2. Swap/Scramble mutation --> Swap 2 values

In [ ]:
def genetic_algorithm(pop_size=10, num_resources=9, num_shifts=3, obj_func=obj.calculate_objective, num_parents=2, selection_mode="proportional", crossover_mode="multi_point", seed=None):
    if seed:
        np.random.seed(seed)
    
    # Initialize the population
    # Note: around half of the population is feasible using range [1, 10]
    population = np.random.randint(1, 10, size=(pop_size, num_resources, num_shifts))
    
    # Fitness evaluation
    parents = []
    obj_costs = []
    for individual in population:
        obj_cost, not_served, wait_exceeded = obj_func(individual)
        if not not_served and not wait_exceeded:
            parents.append(individual)
            obj_costs.append(obj_cost)
            
    # Check for feasibility
    if len(parents) < 2:
        raise Exception("Error: Not enough feasible parents")
    
    """ Natural Selection """
    
    # Convert to numpy arrays
    parents = np.array(parents)
    obj_costs = np.array(obj_costs)
    fitness = 1/np.array(obj_costs)
    chosen_parents = []
    
    # Selection chance proportional to fitness function
    if selection_mode == "proportional":
        relative_fitness = fitness / np.sum(fitness)
        for child_idx in range(pop_size):
            parent_idx = np.random.choice(np.arange(len(parents)), size=num_parents, p=relative_fitness)    # With replacement
            chosen_parents.append(parents[parent_idx, :, :])
    
    # Selection chance proportional to the difference over the worst fitness
    elif selection_mode == "weighted_relative":
        fitness -= np.min(fitness)
        relative_fitness = fitness / np.sum(fitness)
        for child_idx in range(pop_size):
            parent_idx = np.random.choice(np.arange(len(parents)), size=num_parents, p=relative_fitness)    # With replacement
            chosen_parents.append(parents[parent_idx, :, :])
    
    # Selection chance proportional to rank
    elif selection_mode == "rank_based":
        rank = np.argsort(fitness)
        relative_fitness = rank / np.sum(rank)
        for child_idx in range(pop_size):
            parent_idx = np.random.choice(np.arange(len(parents)), size=num_parents, p=relative_fitness)    # With replacement
            chosen_parents.append(parents[parent_idx, :, :])
    
    # Randomly select k1 parents then select num_parents best parents among them
    elif selection_mode == "tournament":
        k1 = 0.5
        k1_size = int(np.ceil(k1 * len(parents)))
        for child_idx in range(pop_size):
            k1_parent_idx = np.random.choice(np.arange(len(parents)), size=k1_size, replace=False)  # Without replacement
            k1_fitness = fitness[k1_parent_idx]
            parent_idx = k1_parent_idx[np.argsort(k1_fitness)[:num_parents]]
            chosen_parents.append(parents[parent_idx, :, :])
    
    # Final shape (pop_size, num_parents, num_resources, num_shifts)
    chosen_parents = np.array(chosen_parents)
    print(chosen_parents)
            
    """ Crossover """
    
    # children = np.zeros((pop_size, num_resources, num_shifts))
    
    # if crossover_mode == "multi_point":
    #     for gene_idx in range(num_resources):
    #         children[:, gene_idx, :] = parents[parent_idx, gene_idx, :]
            
    
genetic_algorithm(selection_mode="tournament", crossover_mode="multi_point")